# Sales Prediction and Customer Analysis Using Supervised Learning Methods
By: Ashley Yuan <br>
NetID: ay362 <br>
Section: 08 

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier

from sqlalchemy import create_engine, text

import sqlite3

## Superstore Sales Dataset

In [2]:
# Read in the data
sales = pd.read_csv('/home/ay362/cs210/Course Project - Final Submissions/train.csv')
sales.head(20)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680
5,6,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.8600
6,7,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.2800
7,8,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.1520
8,9,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.5040
9,10,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9000


In [3]:
sales.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales'],
      dtype='object')

In [4]:
sales.shape

(9800, 18)

In [5]:
sales.describe

<bound method NDFrame.describe of       Row ID        Order ID  Order Date   Ship Date       Ship Mode  \
0          1  CA-2017-152156  08/11/2017  11/11/2017    Second Class   
1          2  CA-2017-152156  08/11/2017  11/11/2017    Second Class   
2          3  CA-2017-138688  12/06/2017  16/06/2017    Second Class   
3          4  US-2016-108966  11/10/2016  18/10/2016  Standard Class   
4          5  US-2016-108966  11/10/2016  18/10/2016  Standard Class   
...      ...             ...         ...         ...             ...   
9795    9796  CA-2017-125920  21/05/2017  28/05/2017  Standard Class   
9796    9797  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   
9797    9798  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   
9798    9799  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   
9799    9800  CA-2016-128608  12/01/2016  17/01/2016  Standard Class   

     Customer ID     Customer Name    Segment        Country             City  \
0       CG-12520    

In [6]:
# Get an idea of the values in our categorical features
print(set(sales['Segment']))
print(set(sales['State']))
print(set(sales['Sub-Category']))

# How many total products does this store sell?
print(len(set(sales['Product Name'])))

# How many total orders were there?
print(len(set(sales['Order ID'])))

# How many different cities were shipped to?
print(len(set(sales['City'])))

# How many customers were there?
print(len(set(sales['Customer ID'])))

{'Home Office', 'Consumer', 'Corporate'}
{'New Mexico', 'Vermont', 'Minnesota', 'Nebraska', 'Illinois', 'Indiana', 'Arkansas', 'Tennessee', 'Delaware', 'Kentucky', 'Massachusetts', 'Idaho', 'Washington', 'Texas', 'Rhode Island', 'Connecticut', 'Ohio', 'Montana', 'Pennsylvania', 'Wisconsin', 'Colorado', 'Kansas', 'Oregon', 'North Dakota', 'West Virginia', 'Oklahoma', 'Utah', 'New York', 'Missouri', 'New Jersey', 'Arizona', 'Maryland', 'Georgia', 'District of Columbia', 'Iowa', 'Nevada', 'South Carolina', 'Florida', 'Louisiana', 'South Dakota', 'Wyoming', 'Michigan', 'Mississippi', 'North Carolina', 'Alabama', 'Virginia', 'New Hampshire', 'California', 'Maine'}
{'Phones', 'Paper', 'Tables', 'Appliances', 'Binders', 'Bookcases', 'Accessories', 'Storage', 'Copiers', 'Supplies', 'Art', 'Furnishings', 'Fasteners', 'Chairs', 'Labels', 'Machines', 'Envelopes'}
1849
4922
529
793


In [ ]:
# Create a bar graph
sales_by_subcategory = sales.groupby('Sub-Category')['Sales'].sum().reset_index()

plt.figure(figsize=(10, 6))
plt.bar(sales_by_subcategory['Sub-Category'], sales_by_subcategory['Sales'], color='paleturquoise')
plt.title('Total Sales by Sub-Category')
plt.xlabel('Sub-Category')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
sales_by_subcategory = sales.groupby('Sub-Category')['Sales'].count().reset_index()

plt.figure(figsize=(10, 6))
plt.bar(sales_by_subcategory['Sub-Category'], sales_by_subcategory['Sales'], color='thistle')
plt.title('Total Sales Counts by Sub-Category')
plt.xlabel('Sub-Category')
plt.ylabel('Total Sales Counts')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Convert the Date column to datetime format
sales['Date'] = pd.to_datetime(sales['Order Date'], format='%d/%m/%Y')

sales_by_date_subcategory = sales.groupby(['Date', 'Sub-Category'])['Sales'].count().reset_index()

# Create a line plot for each sub-category
plt.figure(figsize=(12, 8))

for subcategory in sales_by_date_subcategory['Sub-Category'].unique():
    subset = sales_by_date_subcategory[sales_by_date_subcategory['Sub-Category'] == subcategory]
    plt.plot(subset['Date'], subset['Sales'], marker='o', label=subcategory)

plt.title('Sales Counts by Sub-Category Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales Counts')
plt.xticks(rotation=315)
plt.legend(title='Sub-Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.show()

In [ ]:
# Only 2017 because the previous graph seems too crowded
sales['Date'] = pd.to_datetime(sales['Order Date'], format='%d/%m/%Y')

sales['Month'] = sales['Date'].dt.month

# Filter for sales in the year 2017
sales_2017 = sales[sales['Date'].dt.year == 2017]

sales_by_date_subcategory = sales_2017.groupby(['Month', 'Sub-Category']).count().reset_index()


# Create a line plot for each sub-category
plt.figure(figsize=(12, 8))

for subcategory in sales_by_date_subcategory['Sub-Category'].unique():
    subset = sales_by_date_subcategory[sales_by_date_subcategory['Sub-Category'] == subcategory]
    plt.plot(subset['Month'], subset['Sales'], marker='o', label=subcategory)

plt.title('Sales Counts by Sub-Category in 2017')
plt.xlabel('Month')
plt.ylabel('Total Sales Counts')
plt.legend(title='Sub-Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.show()

In [11]:
# Find missing value counts for each column
for col in sales.columns:
    print(f"Number of missing values in {col}: {sales[col].isnull().sum()}")
print("\n")

Number of missing values in Row ID: 0
Number of missing values in Order ID: 0
Number of missing values in Order Date: 0
Number of missing values in Ship Date: 0
Number of missing values in Ship Mode: 0
Number of missing values in Customer ID: 0
Number of missing values in Customer Name: 0
Number of missing values in Segment: 0
Number of missing values in Country: 0
Number of missing values in City: 0
Number of missing values in State: 0
Number of missing values in Postal Code: 11
Number of missing values in Region: 0
Number of missing values in Product ID: 0
Number of missing values in Category: 0
Number of missing values in Sub-Category: 0
Number of missing values in Product Name: 0
Number of missing values in Sales: 0
Number of missing values in Date: 0
Number of missing values in Month: 0




In [12]:
sales_df = sales.dropna()

In [13]:
# Change Sales into a binary variable
sales_threshold = sales_df['Sales'].median()
sales_df['Sales Class'] = (sales_df['Sales'] > sales_threshold).astype(int)  # 1 for high sales, 0 for low sales

# Code new features
sales_df['Day'] = sales_df['Date'].dt.day
sales_df['Month'] = sales_df['Date'].dt.month
sales_df['Quarter'] = sales_df['Date'].dt.quarter
sales_df['Day of Week'] = sales_df['Date'].dt.dayofweek

# One hot encoding for nominal features 
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded = onehot_encoder.fit_transform(sales_df[['Segment', 'Sub-Category', 'Ship Mode', 'State']])
onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(['Segment', 'Sub-Category', 'Ship Mode', 'State']))
onehot_df
sales_encoded = pd.concat([sales_df[['Order ID', 'Date', 'Day','Month', 'Day of Week', 'Sales Class']].reset_index(drop=True), onehot_df.reset_index(drop=True)],axis=1)

sales_encoded.head(20)

/tmp/ipykernel_1625240/1960559806.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Sales Class'] = (sales_df['Sales'] > sales_threshold).astype(int)  # 1 for high sales, 0 for low sales
/tmp/ipykernel_1625240/1960559806.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_df['Day'] = sales_df['Date'].dt.day
/tmp/ipykernel_1625240/1960559806.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

,Order ID,Date,Day,Month,Day of Week,Sales Class,Segment_Consumer,Segment_Corporate,Segment_Home Office,Sub-Category_Accessories,...,State_South Carolina,State_South Dakota,State_Tennessee,State_Texas,State_Utah,State_Virginia,State_Washington,State_West Virginia,State_Wisconsin,State_Wyoming
0,CA-2017-152156,2017-11-08,8,11,2,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CA-2017-152156,2017-11-08,8,11,2,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,CA-2017-138688,2017-06-12,12,6,0,0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,US-2016-108966,2016-10-11,11,10,1,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,US-2016-108966,2016-10-11,11,10,1,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,CA-2015-115812,2015-06-09,9,6,1,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,CA-2015-115812,2015-06-09,9,6,1,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,CA-2015-115812,2015-06-09,9,6,1,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,CA-2015-115812,2015-06-09,9,6,1,0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,CA-2015-115812,2015-06-09,9,6,1,1,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Decision Tree Model

In [14]:
X = sales_encoded.drop(['Sales Class', 'Order ID', 'Date'], axis=1)
y = sales_encoded['Sales Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Initialize and train the decision tree
model = DecisionTreeClassifier(max_depth=6, min_samples_split=50, random_state=100)
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate the model
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))


Accuracy: 0.7553626149131767
              precision    recall  f1-score   support

           0       0.71      0.87      0.78       984
           1       0.83      0.64      0.72       974

    accuracy                           0.76      1958
   macro avg       0.77      0.75      0.75      1958
weighted avg       0.77      0.76      0.75      1958



In [ ]:
plt.figure(figsize=(70,20))
plot_tree(model, feature_names=X.columns, class_names=['Low Sales', 'High Sales'], filled=True, fontsize=20)
plt.show()

## Random Forest Model with State

In [16]:
# Initialize and train the random forest
rf_model = RandomForestClassifier(n_estimators=200, max_depth=9, min_samples_split=30, random_state=100)
rf_model.fit(X_train, y_train)

# Predict on test data
y_pred = rf_model.predict(X_test)

# Evaluate the model
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Accuracy: 0.7849846782431052
              precision    recall  f1-score   support

           0       0.76      0.84      0.80       984
           1       0.82      0.73      0.77       974

    accuracy                           0.78      1958
   macro avg       0.79      0.78      0.78      1958
weighted avg       0.79      0.78      0.78      1958



In [ ]:
# Get feature importance scores and sort them
feature_importances = pd.Series(rf_model.feature_importances_, index=X.columns).sort_values(ascending=False)
pd.set_option('display.max_rows', 75)
print(feature_importances)

feature_importances[:15].plot(kind='barh')
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Top 15 Features Influencing Sales Prediction (rf_model)")
plt.show()

## GDP Datasets

In [18]:
# Scrape GDP data
url_2015 = 'https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_GDP&oldid=724337497'
url_2016 = 'https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_GDP&oldid=818507395'
url_2017 = 'https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_GDP&oldid=891183538'
url_2018 = 'https://en.wikipedia.org/w/index.php?title=List_of_U.S._states_and_territories_by_GDP&oldid=928295770'


def get_table(url, table_num):
    table = pd.read_html(url)
    data = table[table_num]
    return data

gdp2015 = get_table(url_2015, 2) 
print(gdp2015.head())

gdp2016 = get_table(url_2016, 1)
print(gdp2016.head())

gdp2017 = get_table(url_2017, 3)
print(gdp2017.head())

gdp2018 = get_table(url_2018, 3)
print(gdp2018.head())

   Rank       State     2015       2014
0   1.0  California  2448467  2305921.0
1   2.0       Texas  1639375  1641044.0
2   3.0    New York  1455568  1395488.0
3   4.0     Florida   893189   838939.0
4   5.0    Illinois   771896   736285.0
  Rank Rank (50  states) State or territory           2016 % of Nation
0    —                 —      United States  18,511,499[B]   100.00[B]
1    1                 1         California        2622731       14.17
2    2                 2              Texas        1599283        8.64
3    3                 3           New York        1500055        8.10
4    4                 4            Florida         926049        5.00
  Rank State federal district  or territory           2017 % of Nation  \
0    —                        United States  19,263,350[B]   100.00[B]   
1    1                           California        2746873       14.17   
2    2                                Texas        1696206        8.64   
3    3                             New

In [19]:
print(gdp2016.columns)
print(gdp2017.columns)
print(gdp2018.columns)

Index(['Rank', 'Rank (50  states)', 'State or territory', '2016',
       '% of Nation'],
      dtype='object')
Index(['Rank', 'State federal district  or territory', '2017', '% of Nation',
       'Region'],
      dtype='object')
Index(['Rank', 'State federal district  or territory', '2018 [note 1]',
       '% of Nation', 'Region'],
      dtype='object')


## Database

In [20]:
conn = sqlite3.connect('sales.db')
print("Opened database successfully");

Opened database successfully


In [21]:
cur = conn.execute('''
    CREATE TABLE IF NOT EXISTS Sales (
    `Row ID` varchar(20) PRIMARY KEY,
    `Order ID` varchar(20),
    `Order Date` varchar(20),
    `Ship Date` varchar(20), 
    `Ship Mode` varchar(20),
    `Customer ID` varchar(20), 
    `Customer Name` varchar(20), 
    `Segment` varchar(20), 
    `Country` varchar(20), 
    `City` varchar(20), 
    `State` varchar(20),
    `Postal Code` varchar(20), 
    `Region` varchar(20), 
    `Product ID` varchar(20), 
    `Category` varchar(20), 
    'Sub-Category' varchar(20),
    `Product Name` varchar(20), 
    `Sales` varchar(20)
    
    );
''')

print("Table created successfully");

Table created successfully


In [22]:
cur = conn.execute('''SELECT * FROM Sales LIMIT 5;''')
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'CA-2017-152156', '08/11/2017', '11/11/2017', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420.0, 'South', 'FUR-BO-10001798', 'Furniture', 'Bookcases', 'Bush Somerset Collection Bookcase', 261.96, '2017-11-08 00:00:00.000000', 11)
(2, 'CA-2017-152156', '08/11/2017', '11/11/2017', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420.0, 'South', 'FUR-CH-10000454', 'Furniture', 'Chairs', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 731.94, '2017-11-08 00:00:00.000000', 11)
(3, 'CA-2017-138688', '12/06/2017', '16/06/2017', 'Second Class', 'DV-13045', 'Darrin Van Huff', 'Corporate', 'United States', 'Los Angeles', 'California', 90036.0, 'West', 'OFF-LA-10000240', 'Office Supplies', 'Labels', 'Self-Adhesive Address Labels for Typewriters by Universal', 14.62, '2017-06-12 00:00:00.000000', 6)
(4, 'US-2016-108966', '11/10/2016', '18/10/2016', 'Standard Class', 'SO-20335

In [23]:
cur = conn.execute('''
CREATE TABLE IF NOT EXISTS gdp2015 (
    Rank varchar(20) PRIMARY KEY,
    State varchar(20),
    `2015` varchar(20),
    `2014` varchar(20)
);
''')

print("Table created successfully");

Table created successfully


In [24]:
cur = conn.execute('''
CREATE TABLE IF NOT EXISTS gdp2016 (
    Rank varchar(20) PRIMARY KEY,
    `Rank (50 states)` varchar(20),
    `State or territory` varchar(20),
    `2016` varchar(20),
    `% of Nation` varchar(20)
);
''')


print("Table created successfully");

Table created successfully


In [25]:
cur = conn.execute('''
CREATE TABLE IF NOT EXISTS gdp2017 (
    Rank varchar(20) PRIMARY KEY,
    `State federal district  or territory` varchar(20),
    `2017` varchar(20),
    `% of Nation` varchar(20),
    Region varchar(20)
);
''')
print("Table created successfully")

Table created successfully


In [26]:
cur = conn.execute('''
CREATE TABLE IF NOT EXISTS gdp2018 (
    Rank varchar(20) PRIMARY KEY,
    `State federal district  or territory` varchar(20),
    `2018 [note 1]` varchar(20),
    `% of Nation` varchar(20),
    Region varchar(20)
);
''')

print("Table created successfully");

Table created successfully


In [27]:
# Check if tables are in the database
def tables_in_db(conn):
    cur = conn.execute('''SELECT name FROM sqlite_master WHERE type = 'table';''')
    rows = cur.fetchall()
    
    for row in rows:
        print(row) 
tables_in_db(conn)

('Sales',)
('gdp2015',)
('gdp2016',)
('gdp2017',)
('gdp2018',)


In [28]:
# Import the dataframes to the tables in the database
database_file = 'sales.db'
engine = create_engine(f'sqlite:///{database_file}')

sales.to_sql('Sales', con=engine, if_exists='replace', index=False)
gdp2015.to_sql('gdp2015', con=engine, if_exists='replace', index=False)
gdp2016.to_sql('gdp2016', con=engine, if_exists='replace', index=False)
gdp2017.to_sql('gdp2017', con=engine, if_exists='replace', index=False)
gdp2018.to_sql('gdp2018', con=engine, if_exists='replace', index=False)

57

In [29]:
# Double check if all columns were imported correctly
new_table_names = ['Sales', 'gdp2015', 'gdp2016',  'gdp2017',  'gdp2018']
for new_table in new_table_names:
    with engine.connect() as conn:
        result = conn.execute(text(f'SELECT * FROM {new_table} LIMIT 5'))
        print(new_table)
        
        for row in result:
            print(row)
        print('\n' * 2)

Sales
(1, 'CA-2017-152156', '08/11/2017', '11/11/2017', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420.0, 'South', 'FUR-BO-10001798', 'Furniture', 'Bookcases', 'Bush Somerset Collection Bookcase', 261.96, '2017-11-08 00:00:00.000000', 11)
(2, 'CA-2017-152156', '08/11/2017', '11/11/2017', 'Second Class', 'CG-12520', 'Claire Gute', 'Consumer', 'United States', 'Henderson', 'Kentucky', 42420.0, 'South', 'FUR-CH-10000454', 'Furniture', 'Chairs', 'Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back', 731.94, '2017-11-08 00:00:00.000000', 11)
(3, 'CA-2017-138688', '12/06/2017', '16/06/2017', 'Second Class', 'DV-13045', 'Darrin Van Huff', 'Corporate', 'United States', 'Los Angeles', 'California', 90036.0, 'West', 'OFF-LA-10000240', 'Office Supplies', 'Labels', 'Self-Adhesive Address Labels for Typewriters by Universal', 14.62, '2017-06-12 00:00:00.000000', 6)
(4, 'US-2016-108966', '11/10/2016', '18/10/2016', 'Standard Class', 'SO

In [30]:
# SQL join
new_str = """
WITH total_gdp AS (
SELECT
    State, 
    `2015` AS gdp, 
    '2015' AS year 
FROM 
    gdp2015 
WHERE 
    Rank <> '-'
UNION ALL 

SELECT 
    `State or territory` AS State, 
    `2016` AS gdp, 
    '2016' AS year 
FROM
    gdp2016 
WHERE 
    Rank <> '-'
UNION ALL

SELECT 
    `State federal district  or territory` AS State, 
    `2017` AS gdp, 
    '2017' AS year 
FROM 
    gdp2017 
WHERE 
    Rank <> '-'
UNION ALL 

SELECT 
    `State federal district  or territory` AS State, 
    `2018 [note 1]` AS gdp, 
    '2018' AS year 
FROM 
    gdp2018 
WHERE 
    Rank <> '-'
)

SELECT 
    s.*,
    g.gdp
FROM
    sales s 
    INNER JOIN total_gdp g 
        ON substring(s.`Order Date`, -4, 4) = g.year 
        AND s.State = g.State
"""

In [31]:
# Transform back to dataframe
sales_gdp_df = pd.read_sql(new_str, engine)

# See if data was lost
sales_gdp_df.shape

(9800, 21)

In [32]:
# Double check if there are any null values in the new column
print(f"Number of missing values in GDP: {sales_gdp_df['gdp'].isnull().sum()}")

Number of missing values in GDP: 0


## Random Forest Model with GDP

In [33]:
# Change Sales into a binary variable
sales_gdp_df['Sales Class'] = (sales_gdp_df['Sales'] > sales_threshold).astype(int)

# Code new features
sales_gdp_df['Date'] = pd.to_datetime(sales_gdp_df['Order Date'], format='%d/%m/%Y')

sales_gdp_df['Day'] = sales_gdp_df['Date'].dt.day
sales_gdp_df['Month'] = sales_gdp_df['Date'].dt.month
sales_gdp_df['Quarter'] = sales_gdp_df['Date'].dt.quarter
sales_gdp_df['Day of Week'] = sales_gdp_df['Date'].dt.dayofweek

# One hot encoding for nominal features 
onehot_encoder = OneHotEncoder(sparse_output=False)
onehot_encoded = onehot_encoder.fit_transform(sales_gdp_df[['Segment', 'Sub-Category', 'Ship Mode']])
onehot_df = pd.DataFrame(onehot_encoded, columns=onehot_encoder.get_feature_names_out(['Segment', 'Sub-Category', 'Ship Mode']))
onehot_df
sales_gdp_encoded = pd.concat([sales_gdp_df[['Order ID', 'Date', 'Day','Month', 'Day of Week', 'Sales Class', 'gdp']].reset_index(drop=True), onehot_df.reset_index(drop=True)],axis=1)

sales_gdp_encoded.head(20)

,Order ID,Date,Day,Month,Day of Week,Sales Class,gdp,Segment_Consumer,Segment_Corporate,Segment_Home Office,...,Sub-Category_Machines,Sub-Category_Paper,Sub-Category_Phones,Sub-Category_Storage,Sub-Category_Supplies,Sub-Category_Tables,Ship Mode_First Class,Ship Mode_Same Day,Ship Mode_Second Class,Ship Mode_Standard Class
0,CA-2017-152156,2017-11-08,8,11,2,1,202507,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,CA-2017-152156,2017-11-08,8,11,2,1,202507,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,CA-2017-138688,2017-06-12,12,6,0,0,2746873,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,US-2016-108966,2016-10-11,11,10,1,1,926049,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,US-2016-108966,2016-10-11,11,10,1,0,926049,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
5,CA-2015-115812,2015-06-09,9,6,1,0,2448467,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,CA-2015-115812,2015-06-09,9,6,1,0,2448467,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,CA-2015-115812,2015-06-09,9,6,1,1,2448467,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
8,CA-2015-115812,2015-06-09,9,6,1,0,2448467,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,CA-2015-115812,2015-06-09,9,6,1,1,2448467,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [34]:
X = sales_gdp_encoded.drop(['Sales Class', 'Order ID', 'Date'], axis=1)
y = sales_gdp_encoded['Sales Class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

# Initialize and train the random forest
gdp_model = RandomForestClassifier(n_estimators=200, max_depth=9, min_samples_split=30, random_state=100)
gdp_model.fit(X_train, y_train)

# Predict on test data
y_pred = gdp_model.predict(X_test)

# Evaluate the model
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Accuracy: 0.7816326530612245
              precision    recall  f1-score   support

           0       0.77      0.84      0.80      1020
           1       0.80      0.72      0.76       940

    accuracy                           0.78      1960
   macro avg       0.78      0.78      0.78      1960
weighted avg       0.78      0.78      0.78      1960



In [ ]:
# Get feature importance scores and sort them
feature_importances = pd.Series(gdp_model.feature_importances_, index=X.columns).sort_values(ascending=False)
print(feature_importances)

feature_importances[:15].plot(kind='barh')
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Top 15 Features Influencing Sales Prediction (gdp_model)")
plt.show()